# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2024-05-05 17:18:31,749 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")
        
    def mapping_final_angle_notification_handler(self, uuid, char_bytearr):
        global robot_angles, tof1_mapping_readings, tof2_mapping_readings
        string = self.ble.bytearray_to_string(char_bytearr)
        parts = string.split("|")  
        # print(parts)
        robot_angles.append(float(parts[0][4:]))
        tof1_mapping_readings.append(float(parts[1][5:]))
        tof2_mapping_readings.append(float(parts[2][5:]))



    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        # initialize the PID parameters
        self.ble.send_command(CMD.PID_STATUS_INI, "1")
        self.ble.send_command(CMD.SET_ORIENT_PID_VALUES, "|3|0.01|0.2")
        
        # rotation
        self.ble.send_command(CMD.PID_ANGLE_INCREMENT, "")
        
        # send data (only use the front TOF data)
        global robot_angles, tof1_mapping_readings, tof2_mapping_readings
        print("Start Sending Data!")
        self.ble.send_command(CMD.SEND_PID_ANGLE_INCREMENT_DEBUG_READINGS, "1")
        
        while (len(robot_angles) < 18) or (len(tof1_mapping_readings) < 18*10):   # if the data has not been all collected, just wait
            asyncio.run(asyncio.sleep(2))
        
        observations_count=self.config_params["mapper"]["observations_count"] # 18

        sensor_ranges = []
        sensor_bearings = []
        
        # get the unique elements and keep the raw order of the elements
        robot_angles_unique_elements, robot_angles_unique_indices = np.unique(robot_angles, return_index=True)
        robot_angles_sorted_indices = np.argsort(robot_angles_unique_indices)
        robot_angles = robot_angles_unique_elements[robot_angles_sorted_indices]
        
        # take the stable tof readings and keep the avg value as the final readings
        def remove_outliers(readings):
            q1 = np.percentile(readings, 25)
            q3 = np.percentile(readings, 75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            # print(readings)
            filtered_readings = [x for x in readings if lower_bound <= x and x <= upper_bound]
            return filtered_readings

        def calculate_average(readings):
            stable_readings = remove_outliers(readings)
            if stable_readings:
                return np.mean(stable_readings)
            else:
                return None  # Or handle the case where all readings are considered outliers


        tof1_mapping_readings = np.array(tof1_mapping_readings).reshape(18, 10)
        tof2_mapping_readings = np.array(tof2_mapping_readings).reshape(18, 10)
        
        for i in range(np.shape(tof1_mapping_readings)[0]):
            tof1_mapping_readings[i] = calculate_average(tof1_mapping_readings[i])
            tof2_mapping_readings[i] = calculate_average(tof2_mapping_readings[i])

        if (len(np.shape(tof1_mapping_readings)) > 1):
            tof1_mapping_readings = np.mean(tof1_mapping_readings, axis=1)
            tof2_mapping_readings = np.mean(tof2_mapping_readings, axis=1)
            
        tof1_mapping_readings = tof1_mapping_readings.tolist()
        tof2_mapping_readings = tof2_mapping_readings.tolist()
        robot_angles = robot_angles.tolist()
        # re-order the data list: [20, 360] -> [0, 340]
        robot_angles.insert(0, robot_angles[-1])
        robot_angles = robot_angles[:-1]
        tof1_mapping_readings.insert(0, tof1_mapping_readings[-1])
        tof1_mapping_readings = tof1_mapping_readings[:-1]
        tof2_mapping_readings.insert(0, tof2_mapping_readings[-1])
        tof2_mapping_readings = tof2_mapping_readings[:-1]
        robot_angles[0] = 0 # 360 -> 0
        
        for i in range(observations_count):
            sensor_ranges.append(tof1_mapping_readings[i]/1000)
            sensor_bearings.append(robot_angles[i])

        sensor_ranges = np.array(sensor_ranges).reshape(-1, 1)
        sensor_bearings = np.array(sensor_bearings).reshape(-1, 1)
        
        return sensor_ranges,sensor_bearings

In [4]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2024-05-05 17:18:32,455 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:42:30:78:92:49


2024-05-05 17:18:35,485 | INFO     |: Connected to c0:42:30:78:92:49


/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/notebooks/base_ble.py:104: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  self.client.set_disconnected_callback(self.disconnect_handler)


In [5]:
# for collecting the data
robot_angles = []
tof1_mapping_readings = []
tof2_mapping_readings = []

In [6]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

#
# ble.stop_notify(ble.uuid["RX_STRING"])

ble.start_notify(ble.uuid["RX_STRING"], robot.mapping_final_angle_notification_handler)
#

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2024-05-05 17:18:36,564 | INFO     |:  | Number of observations per grid cell: 18
2024-05-05 17:18:36,566 | INFO     |:  | Precaching Views...


/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/localization.py:150: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2024-05-05 17:18:39,303 | INFO     |:  | Precaching Time: 2.736 secs
2024-05-05 17:18:39,306 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-05-05 17:18:39,308 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [7]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()


# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

unit_convert_factor = 0.3048
test_point_coordinates = np.array([5, -3])
converted_test_pt_coordinates = test_point_coordinates * unit_convert_factor
cmdr.plot_gt(converted_test_pt_coordinates[0], converted_test_pt_coordinates[1])
print(f"Ground Truth: ({converted_test_pt_coordinates[0]}, {converted_test_pt_coordinates[1]}, 0)")
# (-3 ft ,-2 ft ,0 deg)
# (0 ft,3 ft, 0 deg)
# (5 ft,-3 ft, 0 deg)
# (5 ft,3 ft, 0 deg)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2024-05-05 17:18:39,564 | INFO     |: Initializing beliefs with a Uniform Distribution
2024-05-05 17:18:39,565 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
Start Sending Data!
2024-05-05 17:19:05,213 | INFO     |: Update Step
2024-05-05 17:19:05,215 | INFO     |:      | Update Time: 0.001 secs
2024-05-05 17:19:05,216 | INFO     |: Bel index     : (8, 0, 6) with prob = 0.6367104
2024-05-05 17:19:05,217 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2024-05-05 17:19:05,217 | INFO     |: Belief        : (0.914, -1.219, -50.000)
Ground Truth: (1.524, -0.9144000000000001, 0)


qt.pointer.dispatch: skipping QEventPoint(id=1 ts=0 pos=0,0 scn=487.388,227.866 gbl=487.388,227.866 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-487.388,-227.866 last=-487.388,-227.866 Δ 487.388,227.866) : no target window


2024-05-05 17:19:28,198 | INFO     |: Disconnected from 1EF8FB8D-1A06-FB11-4002-B52B84BE3598


qt.pointer.dispatch: skipping QEventPoint(id=4 ts=0 pos=0,0 scn=471.179,460.33 gbl=471.179,460.33 Released ellipse=(1x1 ∡ 0) vel=0,0 press=-471.179,-460.33 last=-471.179,-460.33 Δ 471.179,460.33) : no target window
Traceback (most recent call last):
  File "/Users/harrypeng/Desktop/ECE4160/FastRobots-sim-release-release-mirror/src/plotter.py", line 252, in closeEvent
    result = QtGui.QMessageBox.question(self,
AttributeError: module 'pyqtgraph.Qt.QtGui' has no attribute 'QMessageBox'
